In [288]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import datetime

# 1. Basic DataLoading and Filteration

### Step: 1 ==> Read Loan.csv file

In [289]:
loan_df = pd.read_csv('loan.csv')

C:\Users\Ranjan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Step: 2 ==> Make default datatype as string for all the columns to avoid load error

In [290]:
loan_df['TempdataType'] = "str"
customDataType = dict(zip(loan_df.columns,loan_df.TempdataType)) #Create Custom Datatype to avoid "low_memory" Error
loan_df = pd.read_csv('loan.csv',dtype=customDataType) #Reload

### Step: 3 ==> Find all the Columns with 100% NULL and create a Dictionary

In [291]:
#Get Null% in each Column of loan dataframe
nullColumns = pd.DataFrame(round(100*(loan_df.isnull().sum()/len(loan_df.index)), 2), columns=['null%Column'])
#Create a Map Between Columns to Null% in DataFrame
dictColumn2NullPercent = dict(zip(nullColumns.index,nullColumns['null%Column']))

### Step: 4 ==> Remove all the columns from loan df having 100% Null values

In [292]:
#np Vectorized Function to Filter Null Columns from the Loan Dataframe
Func_filterNullCol = np.vectorize(lambda x: True if dictColumn2NullPercent.get(x) < 100 else False)
Func_filterNullCol(np.array(loan_df.columns))
#Drop All the Null COlumns
loan_df = loan_df.loc[:,Func_filterNullCol(np.array(loan_df.columns))]
loan_df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599,5000,5000,4975,36 months,10.65%,162.87,B,B2,...,NaN,May-16,0,1,INDIVIDUAL,0,0,0,0,0
1,1077430,1314167,2500,2500,2500,60 months,15.27%,59.83,C,C4,...,NaN,Sep-13,0,1,INDIVIDUAL,0,0,0,0,0
2,1077175,1313524,2400,2400,2400,36 months,15.96%,84.33,C,C5,...,NaN,May-16,0,1,INDIVIDUAL,0,0,0,0,0
3,1076863,1277178,10000,10000,10000,36 months,13.49%,339.31,C,C1,...,NaN,Apr-16,0,1,INDIVIDUAL,0,0,0,0,0
4,1075358,1311748,3000,3000,3000,60 months,12.69%,67.79,B,B5,...,Jun-16,May-16,0,1,INDIVIDUAL,0,0,0,0,0


### Step: 5 ==> Drop Columns with Single Unique Value as per Metadata Analysis

In [293]:
unncessaryColumns = ['desc', 'title', 'initial_list_status', 'policy_code', 'application_type',
                     'acc_now_delinq', 'delinq_amnt', 'pymnt_plan', 'url', 'emp_title']
loan_df.drop(columns=unncessaryColumns, inplace=True)

# 2. Normalizing All the Columns with Standard Format for Analysis

## 2.1 ``Common Functions``

In [294]:
#All input dates are in MON-YY Format e.g: DEC-11
def changeDate2SequenceOfInt(inputDate):
    if(pd.isnull(inputDate)):
        return 0
    else:
        dt = datetime.datetime.strptime(inputDate,"%b-%y")
        return int(str(dt.year) + str(dt.strftime('%m')))

In [295]:
def createDict(columnName):
    unique_columnValues = loan_df.loc[:,columnName].unique()
    unique_Count = len(unique_columnValues)
    columnName_col2Val = dict(zip(unique_columnValues,np.arange(unique_Count)))
    columnName_val2Col = dict(zip(np.arange(unique_Count), unique_columnValues))
    return columnName_col2Val, columnName_val2Col

In [296]:
def checkNullPerForColumns(columnName):
    print (round(100*(loan_df.loc[:,columnName].isnull().sum()/len(loan_df.loc[:,columnName].index)), 2))

## 2.2 ``Applying Functions and Lambda to Nornmalize Columns``

### ```==> loan_amnt, funded_amnt, funded_amnt_inv, term ```

In [297]:
loan_df['loan_amnt'] = loan_df['loan_amnt'].apply(pd.to_numeric)
loan_df['funded_amnt'] = loan_df['funded_amnt'].apply(pd.to_numeric)
loan_df['funded_amnt_inv'] = loan_df['funded_amnt_inv'].apply(pd.to_numeric)
loan_df['term'] = (loan_df['term'].apply(lambda x: x.split("months")[0])).apply(pd.to_numeric)

### ```==> int_rate ```

In [298]:
loan_df['int_rate'] = round(loan_df['int_rate'].apply(lambda x: (str(x).split("%")[0])).apply(pd.to_numeric),0)

### ```==> installment ```

In [299]:
loan_df['installment'] = round(loan_df['installment'].apply(pd.to_numeric),0)

### ```==> emp_length ```

In [300]:
#Remove > and < Symbol and < 1 Value set to 0
loan_df['emp_length'] = loan_df['emp_length'].apply(lambda x: str(x).split("year")[0].\
                                                    strip().split("+")[0]).apply(lambda x: 0 if str(x).find("<") >=0 else x)

### ```==> home_ownership ```

In [301]:
# Only 3 Records in NONE Category with Full Paid, Hence same record is discarded
loan_df = loan_df[loan_df['home_ownership']  != 'NONE']

In [302]:
home_ownership_col2Val, home_ownership_val2Col = createDict("home_ownership")
loan_df['home_ownership'] = loan_df['home_ownership'].apply(lambda x: home_ownership_col2Val.get(x))

### ```==> verification_status ```

In [303]:
verification_status_col2Val, verification_status_val2Col = createDict("verification_status")
loan_df['verification_status'] = loan_df['verification_status'].apply(lambda x: verification_status_col2Val.get(x))

### ```==> issue_d, earliest_cr_line ```

In [304]:
loan_df['issue_d'] = loan_df['issue_d'].apply(changeDate2SequenceOfInt)
loan_df['earliest_cr_line'] = loan_df['earliest_cr_line'].apply(changeDate2SequenceOfInt)

### ```==> loan_status ```

In [305]:
loan_status_col2Val, loan_status_val2Col = createDict("loan_status")
loan_df['loan_status'] = loan_df['loan_status'].apply(lambda x: loan_status_col2Val.get(x))

### ```==> purpose ```

In [306]:
purpose_col2Val, purpose_val2Col = createDict("purpose")
loan_df['purpose'] = loan_df['purpose'].apply(lambda x: purpose_col2Val.get(x))

### ```==> zip_code ```

In [307]:
loan_df['zip_code'] = loan_df['zip_code'].apply(lambda x: str(x)[0:3])

### ```==> Set Numeric to All Money Related Columns```

In [308]:
columnList = ['delinq_2yrs', 'inq_last_6mths', 'open_acc',  'pub_rec', 'revol_bal', \
              'total_acc','out_prncp', 'out_prncp_inv','total_pymnt','total_pymnt_inv',\
              'total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries',\
              'collection_recovery_fee','last_pymnt_amnt']
loan_df[columnList] = loan_df[columnList].apply(pd.to_numeric)

### ```==> revol_util ```    ```***``` NAN Value Normalization Need to Check

In [309]:
#Setting the default value 100%, Need to check futher 
loan_df.loc[(pd.isna(loan_df['revol_util'])), 'revol_util'] = 100.0
loan_df['revol_util'] = round(loan_df['revol_util'].apply(lambda x: (str(x).split("%")[0])).apply(pd.to_numeric),0)

### ```==> next_pymnt_d, last_pymnt_d, last_credit_pull_d ```

In [310]:
loan_df['next_pymnt_d'] = loan_df['next_pymnt_d'].apply(changeDate2SequenceOfInt)
loan_df['last_pymnt_d'] = loan_df['last_pymnt_d'].apply(changeDate2SequenceOfInt)
loan_df['last_credit_pull_d'] = loan_df['last_credit_pull_d'].apply(changeDate2SequenceOfInt)

### ```==> collections_12_mths_ex_med ```

In [311]:
collections_12_mths_ex_med_col2Val, collections_12_mths_ex_med_val2Col = createDict("collections_12_mths_ex_med")
loan_df['collections_12_mths_ex_med'] = loan_df['collections_12_mths_ex_med'].apply(lambda x: collections_12_mths_ex_med_col2Val.get(x))

### ```==> chargeoff_within_12_mths ```            *** Need to Check for NAN Value Update***````

In [312]:
loan_df.loc[(pd.isna(loan_df['chargeoff_within_12_mths'])),'chargeoff_within_12_mths'] = -1

### ```==> pub_rec_bankruptcies ```

In [313]:
loan_df.loc[(pd.isna(loan_df['pub_rec_bankruptcies'])),'pub_rec_bankruptcies'] = 0